In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

In [7]:
# deleting a column, reordering and then saving
import pandas as pd
import Utils.Reader as Reader

import scipy.sparse as sps
df = pd.read_csv(filepath_or_buffer='../data/rewatches.csv',
                         sep=",",
                         skiprows=1,
                         header=None,
                         dtype={0: int, 1: int,2: int},
                         engine='python')

Reader.df_col_normalize(df,2)
matrix_df=df.sort_values(by=[0,1])
matrix_df.to_csv('../output/out_2.csv', index=False)

In [1]:
# normalizing a column, reordering and then saving
import pandas as pd
import Utils.Reader as Reader

import scipy.sparse as sps
df = pd.read_csv(filepath_or_buffer='../data/rewatches.csv',
                 sep=",",
                 skiprows=1,
                 header=None,
                 dtype={0: int, 1: int,2: int},
                 engine='python')

df[2]= Reader.df_col_normalize(df,2)
df=df.sort_values(by=[0,1])
df.to_csv('../output/out_2.csv', index=False)

In [6]:
# normalizing a column with substitution, reordering and then saving
import pandas as pd
import Utils.Reader as Reader

import scipy.sparse as sps
matrix_df = pd.read_csv(filepath_or_buffer='../output/extended.csv',
                        sep=",",
                        skiprows=1,
                        header=None,
                        dtype={0: int, 1: int, 2: int},
                        engine='python')
matrix_df.columns = ["a","b","data"]
Reader.df_col_normalize(matrix_df,"data",{-1:-0.5,0:0.5})
matrix_df.head()

,a,b,data
0,0,0,-0.5
1,0,1,-0.5
2,0,2,-0.5
3,0,3,-0.5
4,0,4,-0.5


In [2]:
# 1 HOT ENCODER
import Utils.Reader as Reader
Reader.oneHotEncoder(name="Type", colsToDelete=["Data"],colstoOneHot=["Type"],dfPath='../data/data_ICM_length.csv')

In [17]:
# DF SPLITTER
import Utils.Reader as Reader
Reader.df_splitter("Rewatch",[0,1],dtype={0: int, 1: int},name="rewatches_split_",dfPath='../data/rewatches.csv',colstoDefine=["UserID","ItemID","Rewatch"])

In [1]:
# DF PREPROCESSING MODE
# 1-> Displayed (counts times a given Item has been displayed(in impressions list) to the user
# 2-> Extended (adds an interaction (-1) if a User has an item in impression list
# 3-> Count rewatches of each User-Item pair
# 4-> Count all rewatches of each user
import Utils.Reader as Reader
Reader.read_train_csr(matrix_path="../data/interactions_and_impressions.csv",preprocess=5,saving=True)

Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
Preprocessing with mode: 5


Passing through all dataset to gather infos...:   0%|          | 4087/5826506 [00:02<59:29, 1630.97it/s]  


KeyboardInterrupt: 

# Evaluator data

In [3]:
from Utils.Evaluator import EvaluatorHoldout
import Utils.Reader as Reader

from Data_manager.split_functions.split_train_validation_random_holdout import\
    split_train_in_two_percentage_global_sample
URM_all=Reader.read_train_csr()
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)
evaluator_validation=EvaluatorHoldout(URM_test,cutoff_list=[10])



Unique UserID in the URM are 41629
Unique UserID in the URM and ICM are 41629
5826506
EvaluatorHoldout: Ignoring 1548 ( 3.7%) Users that have less than 1 test interactions


# Stacker

In [35]:
import Utils.Reader as Reader
ICM_path='../data/rewatches.csv'

ICM_all = Reader.load_ICM_rewatches(ICM_path)

ICM_combined = Reader.combine(ICM=ICM_all, URM = URM_all)

In [ ]:
#1
# BEST MODEL UNTIL NOW
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Utils.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.Hybrid.ItemUserHybridKNNRecommender import ItemUserHybridKNNRecommender
import Utils.Reader as Reader

recommender_ItemKNNCFCBF = ItemUserHybridKNNRecommender(ICM_combined.T)
recommender_ItemKNNCFCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCFCBF)
result_df

In [ ]:
#2
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
import Utils.Reader as Reader

recommender_ItemKNNCBF = ItemKNNCBFRecommender(URM_train, ICM_all)
recommender_ItemKNNCBF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCBF)
result_df

In [ ]:
#3
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
import Utils.Reader as Reader

recommender_ItemKNNCF = ItemKNNCFRecommender(ICM_combined.T)
recommender_ItemKNNCF.fit()

result_df, _ = evaluator_validation.evaluateRecommender(recommender_ItemKNNCF)
result_df

In [36]:
from Recommenders.RP3betaCBFRecommender import RP3betaCBFRecommender
#4
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
import Utils.Reader as Reader

rp3betaCBF_recommender = RP3betaCBFRecommender(
    URM_train=URM_train,
    ICM_train=ICM_all,
    verbose=False
)

rp3betaCBF_recommender.fit()
result_df, _ = evaluator_validation.evaluateRecommender(rp3betaCBF_recommender)
result_df

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg


EvaluatorHoldout: Processed 40081 (100.0%) in 33.51 sec. Users per second: 1196


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.103014,0.323226,0.320906,0.075354,0.247813,0.405967,NaN,0.155962,0.564407,0.552096,...,0.962814,0.543419,0.962814,0.232773,12.619525,1.000085,0.528929,0.95723,0.190963,0.557015


In [33]:
from Recommenders.FeatureCombinedImplicitALSRecommender import FeatureCombinedImplicitALSRecommender
#6
IALS_recommender = FeatureCombinedImplicitALSRecommender(
    URM_train=URM_train,
    ICM_train=ICM_all,
    verbose=True
)
IALS_recommender.fit(
    factors=500,
    regularization=0.01,
    use_gpu=False,
    iterations=94,
    num_threads=6,
    confidence_scaling=Reader.linear_scaling_confidence,
    **{
        'URM': {"alpha": 50},
        'ICM': {"alpha": 50}
    }
)
result_df, _ = evaluator_validation.evaluateRecommender(IALS_recommender)
result_df


FeatureCombinedImplicitALSRecommender: URM Detected 1 ( 0.0%) users with no interactions.


ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 27968, expected 24507.

In [37]:
from Recommenders.UserRP3betaRecommender import UserRP3betaRecommender
from Recommenders.RP3betaCBFRecommender import RP3betaCBFRecommender
#7
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
import Utils.Reader as Reader

userRp3beta_recommender = UserRP3betaRecommender(
        URM_train=ICM_combined.T,
        verbose=False
    )

userRp3beta_recommender.fit(
    )
result_df, _ = evaluator_validation.evaluateRecommender(userRp3beta_recommender)
result_df


EvaluatorHoldout: Processed 40081 (100.0%) in 26.70 sec. Users per second: 1501


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.962814,0.0,0.962814,0.029263,9.421967,0.995665,0.065872,0.71439,2.39029,0.343082


In [9]:
#5
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

SLIM_recommender = SLIMElasticNetRecommender(
    URM_train=ICM_combined.T,
    verbose=True
)

SLIM_recommender.fit(
    alpha=0.00026894910579512645,
    l1_ratio=0.08074126876487486,
    topK=int(400),
)
result_df, _ = evaluator_validation.evaluateRecommender(SLIM_recommender)
result_df

KeyboardInterrupt: 

# Submission Creator


In [10]:
# SUBMITTING PREDICTION
import Utils.Writer as Writer
from Utils.Writer import NameRecommender

submit= Writer.Writer(NameRecommender=NameRecommender.RP3betaCBFRecommender, URM=URM_all, ICM=ICM_combined)
submit.makeSubmission()

[    0     1     2 ... 41626 41627 41628]


In [ ]:
from Recommenders.GeneralizedMergedHybridRecommender import GeneralizedMergedHybridRecommender

higher_recommender = GeneralizedMergedHybridRecommender(
    URM_train=URM_train,
    recommenders=[
        rp3betaCBF_recommender,
        userRp3beta_recommender
    ],
    verbose=False
)
higher_recommender.fit(
    alphas=[
        0.4443439790958872 ,
        1 - 0.4443439790958872
    ]
)

result_df, _ = evaluator_validation.evaluateRecommender(higher_recommender)
result_df

/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:276: RuntimeWarning: overflow encountered in power
  return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(scores.shape[0], dtype=np.float64) + 2)),
/Users/fasa/PycharmProjects/RecSys-Challenge-2022-Polimi/Utils/metrics.py:270: RuntimeWarning: invalid value encountered in double_scalars
  ndcg_ = rank_dcg / ideal_dcg
